In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from keras.datasets import imdb
from keras.preprocessing import sequence

In [2]:
# Load and preprocess the data
vocab_size = 5000
max_words = 400
embedding_dim = 32

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

x_valid, y_valid = x_train[:64], y_train[:64]
x_train_, y_train_ = x_train[64:], y_train[64:]

# Convert data to PyTorch tensors
x_train_ = torch.tensor(x_train_, dtype=torch.long)
y_train_ = torch.tensor(y_train_, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.float32)
x_valid = torch.tensor(x_valid, dtype=torch.long)
y_valid = torch.tensor(y_valid, dtype=torch.float32)

train_data = TensorDataset(x_train_, y_train_)
test_data = TensorDataset(x_test, y_test)
valid_data = TensorDataset(x_valid, y_valid)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=False)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [3]:
# Define the PyTorch model
class BiLSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # take the last output of the LSTM
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [4]:
# Hyperparameters
hidden_dim = 128
output_dim = 1
learning_rate = 0.001
num_epochs = 10

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Initialize the model, loss function, and optimizer
model = BiLSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            valid_loss += loss.item()

    print(f'Epoch {epoch + 1}, Validation Loss: {valid_loss / len(valid_loader)}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total * 100:.2f}%')

Using device: cuda
Epoch 1, Validation Loss: 0.5844866037368774
Epoch 2, Validation Loss: 0.5991145372390747
Epoch 3, Validation Loss: 0.3796665072441101
Epoch 4, Validation Loss: 0.28886598348617554
Epoch 5, Validation Loss: 0.5035485625267029
Epoch 6, Validation Loss: 0.3269736170768738
Epoch 7, Validation Loss: 0.292816162109375
Epoch 8, Validation Loss: 0.2389085441827774
Epoch 9, Validation Loss: 0.198494091629982
Epoch 10, Validation Loss: 0.21582314372062683
Accuracy: 83.97%
